In [0]:
%run "/Workspace/Users/dimaklimenko90@gmail.com/databricks_bootcamp_2026/script/silver/crm/silver_crm_cust_info"


In [0]:


df = spark.table("workspace.bronze.crm_cust_info")

df = rename_columns(df)          
df = cast_columns(df)
df = trim_string_columns(df)
df = standardize_categorical(df)



# Check null business key
df_null_keys = df.filter(col("customer_key").isNull())
display(df_null_keys)


# Check duplicates keys
df_duplicates = (df.groupBy("customer_key")
                .count()
                .filter(col("count") > 1)
)



df_duplicates_rows = df.join(
    df_duplicates.select("customer_key"),
    on="customer_key",
    how = "inner"
)
display(df_duplicates_rows.orderBy("customer_key"))




w = Window.partitionBy("customer_key") \
          .orderBy(F.col("create_date").desc())


df_ranked = df.withColumn(
    "rn",
    F.row_number().over(w)
)

df_silver = df_ranked \
    .filter(F.col("rn") == 1) \
    .drop("rn")


df_silver.groupBy("customer_key") \
         .count() \
         .filter(F.col("count") > 1) \
         .show()

